# LangChain의 구성요소

1. `Model` IO은 input과 output으로 나누어져 있습니다.  입력(input)은 prompt(명령을 내리는 곳)입니다. 
    * prompts
    * Language Models
    * Output parser 

2. `Retrieval`은 외부 데이터로 접근하여 이를 모델에 어떻게 제공하는지에 관련된 것입니다.
    * document loaders
    * Transformers
    * Text Embedding 
    * Vector Stores
    * Retrievers 

3. `Chains` - 이전에 조금 해봤던 부분

4. `Agents`는 독립적으로 AI가 작동하도록 만들 수 있게 해주는 기능 (Chains이 필요한 도구들을 선택하여 사용할 수 있도록). 커스텀 도구를 만들어 준다면 chains 스스로 사용할 tool들을 선택할 수 있습니다. 

5. `Memory`는 그냥 챗봇이 기억할 수 있도록 해주는 기능 

6. `callbacks`은 기본적으로 model이 무엇을 하고있는지 중간에 알 수 있도록. 모델이 답변을 제공하기 전에 실제로 모델이 어떤 일을 하고 있는 지 확인할 수 있습니다. 

## FewShotPromptTemplate 

- Fewshot은 모델에게 예제들을 준다는 뜻과 같습니다. 더 나은 대답을 할 수 있도록 하는 예제들을 더 줄 수도 있죠. 
- 구체적으로 대답하는 AI 모델이 필요하다고 가정해봅시다. 모델에게 어떻게 대답해야 하는지에 대한 예제를 AI 모델에게 주는 것이 prompt를 사용해서 어떻게 대답해야 하는지 알려주는 것보다 훨씬 좋습니다. 
    예를 들어 모델에게 콤마를 써서 구분해달라고 하거나, 소문자만 써야한다거나 등등 처럼 이렇게 prompt로 주는 것보다 내가 원하는 것을 예제로 보여주는 것이 더 성공적입니다. 왜냐하면 모델이 텍스트를 만들기 때문이죠. 따라서 prompt로 명령하는 것보다 어떻게 대답해야하는지에 대한 예시를 보여주는 것이 더 나은 방법입니다. 이게 fewShotPromptTemplate가 하는 일입니다. 


이를 통해 예제를 형식화 할 수 있습니다. 예를 들어 고객지원 봇을 만든다고 해볼게요. 회사가 고객 게시판 같이 이미 고객 지원을 위한 시스템이 있다고 했을 때, 그곳에는 많은 고객들과의 대화 기록이나 다른 많은 기록들이 있을 것입니다. 그럼 language model에게 어떻게 고객들에게 대응하는지 알려주고 싶을 것입니다. 그럼 그냥 대화 기록 같은 것들을 데이터베이스에서 가져와서 FewShotPromptTemplate를 사용하여 형식화 시켜주면 더 빠르게 잘 만들 수 있을 것입니다. 이 방식이 기본적으로 언어 모델에게 예제를 주는 방식입니다.  

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}\n") # examples와 동일한 변수로 입력해야함

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix= "Human: What do you know about {country}?", # 위의 형식화가 끝나고 마지막에 실행 사용자의 질문이 어떻게 보여질지
    input_variables=["country"]    
)

chain = prompt | chat 

chain.invoke({
    'country': 'korea'
})                           # 결과가 예시와 같게 출력됨.

AI: 
I know this:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won

AIMessageChunk(content='AI: \nI know this:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: South Korean Won')